# Data engineering - Linen Assignment (Alexandre Gallego Mato)

## Creating the connection with the SQL server

In [1]:
# Use this one for database access
import pyodbc

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import dates

# Convert columns day/hour/minute/second into datetime
def getTime(row):
    return datetime.strptime(\
        '2019-06-' + str(int(row['day'])) + ' ' + \
        str(int(row['hour'])) + ':' + str(int(row['minute'])) + ':' + str(int(row['second'])),\
        '%Y-%m-%d %H:%M:%S'\
    )

# -------------------------------------------------
# Connect to the database
# -------------------------------------------------        
def connectDB():
    driver = 'SQL Server'
    server = 'ieserver.database.windows.net'
    db = 'LossLessLinenDB'
    user = 'student'
    password = 'IELab01!'
    connection = pyodbc.connect('driver={%s};server=%s;database=%s;uid=%s;pwd=%s' % (driver, server, db, user, password ))
        
    return connection

# -------------------------------------------------
# Return the result of a SQL query as a dataframe
# -------------------------------------------------    
def executeSQL(connection, SQLSentence):
    df = pd.read_sql_query(SQLSentence, connection)
    return df


## Creation of SQL functions in Python

These functions will provide us the following information: number of distinct tag readings per cycle in the **soiled** proces, number of distinct tagreadings per cycle in the **clean** cycle and the number of distinct readings per process selected. This will enable us to compare the total aggregated number of readings per cycle with each of the readings per process. If the difference is higher or equal to 30%, the cycle is a misleading cycle as specified in the statement. For the assignment as is going to be specified below, I have picked the hotel with id **2480**.

In [2]:
# Function to retrieve the total distinct readings per soiled cycle between the dates specified in the where clause

def total_soiled_cycle_readingsQuery(hotelId):
    query = f"""
    SELECT COUNT(DISTINCT tr.TagId)AS differenttagreadings, r.BatchNum, r.FacilityId
    FROM TagReadings tr
    INNER JOIN Readings r ON tr.ReadingId = r.Id
    INNER JOIN ReadingStatus rs ON r.ReadingStatusId = rs.Id
    INNER Join Facilities f ON f.Id = r.FacilityId
    WHERE r.ReadingDate BETWEEN '2020-03-01' AND '2020-03-15'
    AND(f.Id = {hotelId})
    AND (rs.Code BETWEEN 1 AND 4)
    GROUP BY r.BatchNum, r.FacilityId
    """

    return query

In [3]:
# Function to retrieve the total distinct readings pero cleaned cycle between the dates specified in the where clause

def total_clean_cycle_readingsQuery(hotelId):
    query3 = f"""
    SELECT COUNT(DISTINCT tr.TagId)AS differenttagreadings, r.BatchNum, r.FacilityId
    FROM TagReadings tr
    INNER JOIN Readings r ON tr.ReadingId = r.Id
    INNER JOIN ReadingStatus rs ON r.ReadingStatusId = rs.Id
    INNER Join Facilities f ON f.Id = r.FacilityId
    WHERE r.ReadingDate BETWEEN '2020-03-01' AND '2020-03-15'
    AND(f.Id = {hotelId})
    AND (rs.Code IN (5,6,7,0))
    GROUP BY r.BatchNum, r.FacilityId
    """

    return query3

In [4]:
# Function to retrieve the total distinct readings per process between the dates specified in the where clause

def different_processes_readings(hotelId, process_number):
    query2 = f"""
    SELECT COUNT(DISTINCT tr.TagId)AS differenttagreadings{process_number}, r.BatchNum
    FROM TagReadings tr
    INNER JOIN Readings r ON tr.ReadingId = r.Id
    INNER JOIN ReadingStatus rs ON r.ReadingStatusId = rs.Id
    INNER Join Facilities f ON f.Id = r.FacilityId
    WHERE r.ReadingDate BETWEEN '2020-03-01' AND '2020-03-15'
    AND(f.Id = {hotelId})
    AND (rs.Code = {process_number})
    GROUP BY r.BatchNum, r.FacilityId, rs.Code
    """
    return query2
    

## EXTRACTING THE DIFFERENT TAG READINGS FOR THE SOILED CYCLE OF HOTEL 2480

In [5]:
c = connectDB()

total_readings_in_soiled_batch = executeSQL(c, total_soiled_cycle_readingsQuery(2480))

print(total_readings_in_soiled_batch)

    differenttagreadings               BatchNum  FacilityId
0                    178  zz20200301000000S2480        2480
1                    148  zz20200302000000S2480        2480
2                     79  zz20200303000000S2480        2480
3                    143  zz20200304000000S2480        2480
4                    156  zz20200305000000S2480        2480
5                    158  zz20200306000000S2480        2480
6                    125  zz20200307000000S2480        2480
7                    190  zz20200308000000S2480        2480
8                    253  zz20200309000000S2480        2480
9                    147  zz20200310000000S2480        2480
10                   189  zz20200311000000S2480        2480
11                   122  zz20200312000000S2480        2480
12                   185  zz20200313000000S2480        2480
13                   129  zz20200314000000S2480        2480


## EXTRACTING THE READINGS OF DIFFERENT CYCLES IN PROCESS 1 OF HOTEL 2480

In [6]:
readings_of_code_1 = executeSQL(c, different_processes_readings(2480, 1))
print(readings_of_code_1)

    differenttagreadings1               BatchNum
0                      78  zz20200303000000S2480
1                     143  zz20200304000000S2480
2                     153  zz20200305000000S2480
3                     158  zz20200306000000S2480
4                     124  zz20200307000000S2480
5                     171  zz20200308000000S2480
6                     247  zz20200309000000S2480
7                     142  zz20200310000000S2480
8                     185  zz20200311000000S2480
9                     121  zz20200312000000S2480
10                    174  zz20200313000000S2480
11                    129  zz20200314000000S2480


## EXTRACTING THE READINGS OF DIFFERENT CYCLES IN PROCESS 2 OF HOTEL 2480

In [7]:
readings_of_code_2 = executeSQL(c, different_processes_readings(2480, 2))
print(readings_of_code_2)

    differenttagreadings2               BatchNum
0                     168  zz20200301000000S2480
1                     145  zz20200302000000S2480
2                     142  zz20200304000000S2480
3                     150  zz20200305000000S2480
4                     148  zz20200306000000S2480
5                     121  zz20200307000000S2480
6                     244  zz20200309000000S2480
7                     134  zz20200310000000S2480
8                     184  zz20200311000000S2480
9                     119  zz20200312000000S2480
10                    171  zz20200313000000S2480
11                    110  zz20200314000000S2480


## EXTRACTING THE READINGS OF DIFFERENT CYCLES IN PROCESS 3 OF HOTEL 2480

In [8]:
readings_of_code_3 = executeSQL(c, different_processes_readings(2480, 3))
print(readings_of_code_3)

   differenttagreadings3               BatchNum
0                     76  zz20200303000000S2480
1                    137  zz20200305000000S2480


## EXTRACTING THE READINGS OF DIFFERENT CYCLES IN PROCESS 4 OF HOTEL 2480

In [9]:
readings_of_code_4 = executeSQL(c, different_processes_readings(2480, 4))
print(readings_of_code_4)

    differenttagreadings4               BatchNum
0                     177  zz20200301000000S2480
1                     146  zz20200302000000S2480
2                      76  zz20200303000000S2480
3                     149  zz20200306000000S2480
4                     121  zz20200307000000S2480
5                     169  zz20200308000000S2480
6                     227  zz20200309000000S2480
7                     140  zz20200310000000S2480
8                     183  zz20200311000000S2480
9                     113  zz20200312000000S2480
10                    170  zz20200313000000S2480
11                    114  zz20200314000000S2480


## Definition of functions for data preparation

The following functions will do the following: merge the different dataframes of the soiled and cleaned cycle, in order to have all the relevant data within one dataframe to perform the calculations mentioned before. The second function, once the data is prepared, will perform the absolute percentage difference between the total distinct tag-readings in the cycle and the total number of distinct tag-readings of each process. Once we know this difference, the third function will give us the misleading cycles based on the criteria specified previously.

In [10]:
# function to merge all the relevant data within one dataframe

def merge_function(left_dataframe, right_dataframe, column_to_join, join):
    
    merged_dataframe = left_dataframe.merge(right_dataframe, on = column_to_join, how = join)
    
    return merged_dataframe


cycle_readings = merge_function(total_readings_in_soiled_batch, readings_of_code_1, "BatchNum", "left")

cycle_readings = merge_function(cycle_readings, readings_of_code_2, "BatchNum", "left")

cycle_readings = merge_function(cycle_readings, readings_of_code_3, "BatchNum", "left")

cycle_readings = merge_function(cycle_readings, readings_of_code_4, "BatchNum", "left")

cycle_readings = cycle_readings.fillna(0)

#print(cycle_readings)

# function to calculate the absolute percentage difference needed to detect the misleading cycles

def absolute_percentage_difference_dataframe(dataframe, column1, column2, new_column):
    
    dataframe[new_column] = round(abs((dataframe[column1] - dataframe[column2])/dataframe[column1]) * 100,2)
    
    return dataframe

dataframe_complete = absolute_percentage_difference_dataframe(cycle_readings, "differenttagreadings", "differenttagreadings1", "(total_cycle_readings - process1_readings)%")

dataframe_complete = absolute_percentage_difference_dataframe(cycle_readings, "differenttagreadings", "differenttagreadings2", "(total_cycle_readings - process2_readings)%")

dataframe_complete = absolute_percentage_difference_dataframe(cycle_readings, "differenttagreadings", "differenttagreadings3", "(total_cycle_readings - process3_readings)%")

dataframe_complete = absolute_percentage_difference_dataframe(cycle_readings, "differenttagreadings", "differenttagreadings4", "(total_cycle_readings - process4_readings)%")

print (cycle_readings)

misleading_cycles = []

# function to identify the misleading cycles

def list_of_misleading_cycles(dataframe, column_value, column_cycle):
    
    for i in range(0, len(dataframe[column_value])):
        if dataframe[column_value][i] >= 30.00 and dataframe[column_cycle][i] not in misleading_cycles:
            misleading_cycles.append(dataframe[column_cycle][i])
    
    return misleading_cycles

    differenttagreadings               BatchNum  FacilityId  \
0                    178  zz20200301000000S2480        2480   
1                    148  zz20200302000000S2480        2480   
2                     79  zz20200303000000S2480        2480   
3                    143  zz20200304000000S2480        2480   
4                    156  zz20200305000000S2480        2480   
5                    158  zz20200306000000S2480        2480   
6                    125  zz20200307000000S2480        2480   
7                    190  zz20200308000000S2480        2480   
8                    253  zz20200309000000S2480        2480   
9                    147  zz20200310000000S2480        2480   
10                   189  zz20200311000000S2480        2480   
11                   122  zz20200312000000S2480        2480   
12                   185  zz20200313000000S2480        2480   
13                   129  zz20200314000000S2480        2480   

    differenttagreadings1  differenttagreadings2  diff

In [11]:
misleading_cycles_soiled = list_of_misleading_cycles(cycle_readings, "(total_cycle_readings - process1_readings)%", "BatchNum")

misleading_cycles_soiled = list_of_misleading_cycles(cycle_readings, "(total_cycle_readings - process2_readings)%", "BatchNum")

misleading_cycles_soiled = list_of_misleading_cycles(cycle_readings, "(total_cycle_readings - process3_readings)%", "BatchNum")

misleading_cycles_soiled = list_of_misleading_cycles(cycle_readings, "(total_cycle_readings - process4_readings)%", "BatchNum")

print(misleading_cycles_soiled)

print(len(misleading_cycles_soiled))

['zz20200301000000S2480', 'zz20200302000000S2480', 'zz20200303000000S2480', 'zz20200308000000S2480', 'zz20200304000000S2480', 'zz20200306000000S2480', 'zz20200307000000S2480', 'zz20200309000000S2480', 'zz20200310000000S2480', 'zz20200311000000S2480', 'zz20200312000000S2480', 'zz20200313000000S2480', 'zz20200314000000S2480', 'zz20200305000000S2480']
14


## Misleading soiled cycles for hotel 2480

As it can be seen in the following dataframe, every cycle within the soiled cycles has some errors on its readings for the different processes. In this specific case, the process 3 is the one with higher number of misleading readings since only two cycles have a difference less than 30%, the rest of them is of 100%.

In [12]:
soiled_misleading_cycles = pd.DataFrame(misleading_cycles_soiled, columns = ["Misleading soiled cycles"])

print(soiled_misleading_cycles)

   Misleading soiled cycles
0     zz20200301000000S2480
1     zz20200302000000S2480
2     zz20200303000000S2480
3     zz20200308000000S2480
4     zz20200304000000S2480
5     zz20200306000000S2480
6     zz20200307000000S2480
7     zz20200309000000S2480
8     zz20200310000000S2480
9     zz20200311000000S2480
10    zz20200312000000S2480
11    zz20200313000000S2480
12    zz20200314000000S2480
13    zz20200305000000S2480


## CLEAN CYCLES

The same process followed for the soiled cycles is applied to the cleaning cycles.

In [13]:
total_readings_in_clean_batch = executeSQL(c, total_clean_cycle_readingsQuery(2480))

print(total_readings_in_clean_batch)

    differenttagreadings               BatchNum  FacilityId
0                    220  zz20200301000000C2480        2480
1                    169  zz20200302000000C2480        2480
2                    172  zz20200303000000C2480        2480
3                     88  zz20200304000000C2480        2480
4                    166  zz20200305000000C2480        2480
5                    153  zz20200306000000C2480        2480
6                    156  zz20200307000000C2480        2480
7                    298  zz20200308000000C2480        2480
8                    166  zz20200309000000C2480        2480
9                    241  zz20200310000000C2480        2480
10                   151  zz20200311000000C2480        2480
11                   184  zz20200312000000C2480        2480
12                   127  zz20200313000000C2480        2480
13                   190  zz20200314000000C2480        2480


In [14]:
readings_of_code_0 = executeSQL(c, different_processes_readings(2480, 0))
print(readings_of_code_0)

    differenttagreadings0               BatchNum
0                     169  zz20200302000000C2480
1                     144  zz20200303000000C2480
2                      88  zz20200304000000C2480
3                     140  zz20200305000000C2480
4                     153  zz20200306000000C2480
5                     153  zz20200307000000C2480
6                     128  zz20200308000000C2480
7                     162  zz20200309000000C2480
8                     241  zz20200310000000C2480
9                     150  zz20200311000000C2480
10                    184  zz20200312000000C2480
11                    121  zz20200313000000C2480
12                    190  zz20200314000000C2480


In [15]:
readings_of_code_5 = executeSQL(c, different_processes_readings(2480, 5))
print(readings_of_code_5)

    differenttagreadings5               BatchNum
0                     218  zz20200301000000C2480
1                     148  zz20200302000000C2480
2                     166  zz20200303000000C2480
3                      86  zz20200304000000C2480
4                     153  zz20200305000000C2480
5                     151  zz20200306000000C2480
6                     142  zz20200307000000C2480
7                     128  zz20200308000000C2480
8                     155  zz20200309000000C2480
9                     231  zz20200310000000C2480
10                    148  zz20200311000000C2480
11                    180  zz20200312000000C2480
12                    121  zz20200313000000C2480
13                    180  zz20200314000000C2480


In [16]:
readings_of_code_6 = executeSQL(c, different_processes_readings(2480, 6))
print(readings_of_code_6)

   differenttagreadings6               BatchNum
0                    147  zz20200302000000C2480
1                    150  zz20200303000000C2480


In [17]:
readings_of_code_7 = executeSQL(c, different_processes_readings(2480, 7))
print(readings_of_code_7)

    differenttagreadings7               BatchNum
0                     192  zz20200301000000C2480
1                     149  zz20200302000000C2480
2                      85  zz20200304000000C2480
3                     145  zz20200305000000C2480
4                     151  zz20200306000000C2480
5                     148  zz20200307000000C2480
6                     287  zz20200308000000C2480
7                     158  zz20200309000000C2480
8                     182  zz20200310000000C2480
9                     150  zz20200311000000C2480
10                    182  zz20200312000000C2480
11                    119  zz20200313000000C2480
12                    181  zz20200314000000C2480


## Creating the dataframe including all clean processes per cycle

In [18]:
# preparing the dataframe with all the relevant data

cycle_readings_clean = merge_function(total_readings_in_clean_batch, readings_of_code_5, "BatchNum", "left")

cycle_readings_clean = merge_function(cycle_readings_clean, readings_of_code_0, "BatchNum", "left")

cycle_readings_clean = merge_function(cycle_readings_clean, readings_of_code_6, "BatchNum", "left")

cycle_readings_clean = merge_function(cycle_readings_clean, readings_of_code_7, "BatchNum", "left")

cycle_readings_clean = cycle_readings_clean.fillna(0)

print(cycle_readings_clean)


    differenttagreadings               BatchNum  FacilityId  \
0                    220  zz20200301000000C2480        2480   
1                    169  zz20200302000000C2480        2480   
2                    172  zz20200303000000C2480        2480   
3                     88  zz20200304000000C2480        2480   
4                    166  zz20200305000000C2480        2480   
5                    153  zz20200306000000C2480        2480   
6                    156  zz20200307000000C2480        2480   
7                    298  zz20200308000000C2480        2480   
8                    166  zz20200309000000C2480        2480   
9                    241  zz20200310000000C2480        2480   
10                   151  zz20200311000000C2480        2480   
11                   184  zz20200312000000C2480        2480   
12                   127  zz20200313000000C2480        2480   
13                   190  zz20200314000000C2480        2480   

    differenttagreadings5  differenttagreadings0  diff

In [19]:
# calculating the absolute percentage difference

dataframe_complete_1 = absolute_percentage_difference_dataframe(cycle_readings_clean, "differenttagreadings", "differenttagreadings5", "(total_cycle_readings - process5_readings)%")

dataframe_complete_1 = absolute_percentage_difference_dataframe(cycle_readings_clean, "differenttagreadings", "differenttagreadings6", "(total_cycle_readings - process6_readings)%")

dataframe_complete_1 = absolute_percentage_difference_dataframe(cycle_readings_clean, "differenttagreadings", "differenttagreadings7", "(total_cycle_readings - process7_readings)%")

dataframe_complete_1 = absolute_percentage_difference_dataframe(cycle_readings_clean, "differenttagreadings", "differenttagreadings0", "(total_cycle_readings - process0_readings)%")

print(dataframe_complete_1)

print(dataframe_complete_1.columns.to_list())


    differenttagreadings               BatchNum  FacilityId  \
0                    220  zz20200301000000C2480        2480   
1                    169  zz20200302000000C2480        2480   
2                    172  zz20200303000000C2480        2480   
3                     88  zz20200304000000C2480        2480   
4                    166  zz20200305000000C2480        2480   
5                    153  zz20200306000000C2480        2480   
6                    156  zz20200307000000C2480        2480   
7                    298  zz20200308000000C2480        2480   
8                    166  zz20200309000000C2480        2480   
9                    241  zz20200310000000C2480        2480   
10                   151  zz20200311000000C2480        2480   
11                   184  zz20200312000000C2480        2480   
12                   127  zz20200313000000C2480        2480   
13                   190  zz20200314000000C2480        2480   

    differenttagreadings5  differenttagreadings0  diff

In [20]:
# defining the function to calculate the misleading cycles in the cleaning cycles

clean_misleading_cycles = []

def list_of_clean_misleading_cycles(dataframe, column_value, column_cycle):
    
    for i in range(0, len(dataframe[column_value])):
       if dataframe[column_value][i] >= 30.00 and dataframe[column_cycle][i] not in clean_misleading_cycles:
            clean_misleading_cycles.append(dataframe[column_cycle][i])
    
    return clean_misleading_cycles

In [21]:
# retrieval of the misleading cycles

misleading_cycles_cleaned = list_of_clean_misleading_cycles(cycle_readings_clean, "(total_cycle_readings - process5_readings)%", "BatchNum")

misleading_cycles_cleaned = list_of_clean_misleading_cycles(cycle_readings_clean, "(total_cycle_readings - process6_readings)%", "BatchNum")

misleading_cycles_cleaned = list_of_clean_misleading_cycles(cycle_readings_clean, "(total_cycle_readings - process7_readings)%", "BatchNum")

misleading_cycles_cleaned = list_of_clean_misleading_cycles(cycle_readings_clean, "(total_cycle_readings - process0_readings)%", "BatchNum")

print(misleading_cycles_cleaned)

['zz20200308000000C2480', 'zz20200301000000C2480', 'zz20200304000000C2480', 'zz20200305000000C2480', 'zz20200306000000C2480', 'zz20200307000000C2480', 'zz20200309000000C2480', 'zz20200310000000C2480', 'zz20200311000000C2480', 'zz20200312000000C2480', 'zz20200313000000C2480', 'zz20200314000000C2480', 'zz20200303000000C2480']


## Dataframe with the misleading cycles in the cleaning process

In [22]:
cleaning_misleading_cycles = pd.DataFrame(misleading_cycles_cleaned, columns = ["Misleading cleaning cycles"])

print(cleaning_misleading_cycles)

   Misleading cleaning cycles
0       zz20200308000000C2480
1       zz20200301000000C2480
2       zz20200304000000C2480
3       zz20200305000000C2480
4       zz20200306000000C2480
5       zz20200307000000C2480
6       zz20200309000000C2480
7       zz20200310000000C2480
8       zz20200311000000C2480
9       zz20200312000000C2480
10      zz20200313000000C2480
11      zz20200314000000C2480
12      zz20200303000000C2480


## Final conclusion

As it can be seen in this final dataframe, all the cycles within the soiled process for the hotel with id = 2480, had a specific kind of misreading in one of the different processes (1,2,3,4). However, in the cleaning process was different, since one cycle was able to complete all the readings without any significant error (larger than 30%). Taking everything into account, this hotel will need to improve the quality of its readings since most of its cycles had errors in at least one of the processes.

In [23]:
misleading_cycles = soiled_misleading_cycles.merge(cleaning_misleading_cycles, left_index=True, right_index=True, how="left")

print(misleading_cycles)

   Misleading soiled cycles Misleading cleaning cycles
0     zz20200301000000S2480      zz20200308000000C2480
1     zz20200302000000S2480      zz20200301000000C2480
2     zz20200303000000S2480      zz20200304000000C2480
3     zz20200308000000S2480      zz20200305000000C2480
4     zz20200304000000S2480      zz20200306000000C2480
5     zz20200306000000S2480      zz20200307000000C2480
6     zz20200307000000S2480      zz20200309000000C2480
7     zz20200309000000S2480      zz20200310000000C2480
8     zz20200310000000S2480      zz20200311000000C2480
9     zz20200311000000S2480      zz20200312000000C2480
10    zz20200312000000S2480      zz20200313000000C2480
11    zz20200313000000S2480      zz20200314000000C2480
12    zz20200314000000S2480      zz20200303000000C2480
13    zz20200305000000S2480                        NaN
